In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix,
                             classification_report)

# --- 1. Carregamento e Preparação dos Dados ---

# Carregar o dataset que teve tratamento de outliers
try:
    df = pd.read_csv('df_sem_outliers.csv')
except FileNotFoundError:
    print("Arquivo 'df_sem_outliers.csv' não encontrado. Certifique-se de que ele foi salvo pelo notebook 'analise.ipynb'.")
    exit()

print("Formato do DataFrame (sem outliers):", df.shape)
print("\nDistribuição da variável-alvo 'has_failure':")
print(df['has_failure'].value_counts(normalize=True))

# Separar variáveis independentes (X) e dependente (y)
X = df.drop('has_failure', axis=1)
y = df['has_failure']

# Dividir os dados em conjuntos de treino e teste
# Usar stratify=y para manter a proporção da variável-alvo
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\nConjunto de treino: {X_train.shape}")
print(f"Conjunto de teste: {X_test.shape}")


# --- 2. Standardização (Feature Scaling) ---

# Modelos como Regressão Logística e SVM são sensíveis à escala das features.
# É crucial ajustar o scaler APENAS nos dados de treino para evitar vazamento de dados.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# --- 3. Treinamento e Avaliação dos Modelos ---

# Dicionário com os modelos a serem testados
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'SVM': SVC(random_state=42, probability=True) # probability=True para calcular ROC AUC
}

# Loop para treinar e avaliar cada modelo
for name, model in models.items():
    print(f"\n----- Avaliando: {name} -----")
    
    # Treinar o modelo com os dados padronizados
    model.fit(X_train_scaled, y_train)
    
    # Fazer previsões no conjunto de teste padronizado
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] # Probabilidades para AUC
    
    # Calcular e exibir as métricas de avaliação
    print(f"Acurácia: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
    print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
    
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))
    
    print("\nRelatório de Classificação Detalhado:")
    print(classification_report(y_test, y_pred))

Formato do DataFrame (sem outliers): (21022, 52)

Distribuição da variável-alvo 'has_failure':
has_failure
1    0.618447
0    0.381553
Name: proportion, dtype: float64

Conjunto de treino: (14715, 51)
Conjunto de teste: (6307, 51)

----- Avaliando: Logistic Regression -----
Acurácia: 0.6329
Precisão: 0.6451
Recall: 0.9036
F1-Score: 0.7528
ROC AUC: 0.6306

Matriz de Confusão:
[[ 467 1939]
 [ 376 3525]]

Relatório de Classificação Detalhado:
              precision    recall  f1-score   support

           0       0.55      0.19      0.29      2406
           1       0.65      0.90      0.75      3901

    accuracy                           0.63      6307
   macro avg       0.60      0.55      0.52      6307
weighted avg       0.61      0.63      0.58      6307


----- Avaliando: SVM -----
Acurácia: 0.6475
Precisão: 0.6548
Recall: 0.9098
F1-Score: 0.7615
ROC AUC: 0.6501

Matriz de Confusão:
[[ 535 1871]
 [ 352 3549]]

Relatório de Classificação Detalhado:
              precision    recal